In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))
# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


/var/folders/8t/f5lr_py56_78f39pmpx4l3sm0000gn/T/ipykernel_1338/1949893161.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


# CMPS 2200
# Introduction to Algorithms

##  SPARC and Recurrence


### Recursive Algorithms for Parallelism

> We calculate work $W = T_1$ and span $S = T_\infty$. This is also defined as **Work-Span Model** to analyze parallel algorithms.

> Parallelism $\dfrac{W}{S}$.

### Work Efficiency

> A parallel algorithm is (asymptotically) **work efficient** if the work is asymptotically the same as the time for an optimal sequential algorithm that solves the same problem.

### Amdahl's Law


$$\dfrac{T_1}{T_p} = \frac{1}{S + \dfrac{1 − S}{p}} < \frac{1}{S}, \text{where $S$ is the amount of time that cannot be parallelized.}$$

### Greedy Schedler 


<a href="https://stanford.edu/~rezab/dao/notes/lecture01/cme323_lec1.pdf">**Brent's Theorem**</a>: When we only have $P$ processors, the time $T_P$ to perform computation is bounded by:

$$T_P < \frac{T_1}{P} + T_\infty = \frac{W}{P} + S $$ 


<br>

$$\Rightarrow T_P \in \mathcal{O}(\frac{W}{P} + S)$$


<br>
<br>


Because we know:  
- $T_P \ge \frac{W}{P}$, since that would be the optimal division of work to processors
- $T_P \ge S = T_\infty$, by the definition of span

we can conclude that the best we can hope for is:

$$T_P \ge \mathrm{max}(\frac{W}{P},S)$$

Therefore, the time using a greedy scheduler is bounded by:

$$ \mathrm{max}(\frac{W}{P},S) \le T_P < \frac{W}{P} + S$$

<br>
How good is greedy? How close is $(\frac{W}{P} + S)$ to $\mathrm{max}(\frac{W}{P},S)$?

Recall parallelism: $\overline{P} = \frac{W}{S}$. So, **the greater $\overline{P}$ is than $P$**, the closer to optimal we get.

$$\max(\frac{W}{P}, S) = \frac{W}{P}$$

$$\frac{W}{P} + S = \frac{W}{P}+\frac{W}{\overline{P}} = \frac{W}{P}(1+\frac{P}{\overline{P}}) ~~~\approx \frac{W}{P}$$




## Work-Span Model

For a given expression $e$, $\color{blue}{\text{a series of statements or a sequence of instructions}}$, we will analyze the work $W(e)$ and span $S(e)$ 

All we have seen before is Python code.


```python

# recursive, serial
def sum_list_recursive(mylist):
    print('summing %s' % mylist)
    if len(mylist) == 1:
        return mylist[0]
    return (
        sum_list_recursive(mylist[:len(mylist)//2]) +
        sum_list_recursive(mylist[len(mylist)//2:])
    )
```



```python


# recursive, parallel

from multiprocessing.pool import ThreadPool

def in_parallel(f1, arg1, f2, arg2):
    with ThreadPool(2) as pool:
        result1 = pool.apply_async(f1, [arg1])  # launch f1
        result2 = pool.apply_async(f2, [arg2])  # launch f2
        return (result1.get(), result2.get())   # wait for both to finish

def sum_list_recursive_parallel(mylist):
    result1, result2 = in_parallel(
        sum_list_recursive_parallel, mylist[:len(mylist)//2],
        sum_list_recursive_parallel, mylist[len(mylist)//2:]
    )
    # combine results
    return result1 + result2


```

## SPARC

Our textbook uses a **''pseudo code''** language called **SPARC**
- based on [Standard ML](https://en.wikipedia.org/wiki/Standard_ML) [ML: Meta language]
- functional language

When possible, we will also show Python versions of key algorithms.




## Example SPARC program


<br><br>
<p> <span>\[\begin{array}{l}  
\texttt{let}\\   
~~~~x = 2 + 3\\  
~~~~f (w) = (w * 4, w - 2)\\  
~~~~(y,z) = f(x-1)\\  
\texttt{in}\\   
~~~~x + y + z\\  
\texttt{end}   
\end{array}\]</span></p>
<br><br>

<br><br>
**binding**: associate entities (data or code) with identifiers.

<br>

**let expression:**

**let**  
$\:\: b^+$  
**in**  
$\:\:e$  
**end**

Expression $e$ is applied using the bindings defined inside **let**.

<br><br>
**expression** *e*: describes a computation  
- **evaluating** an expression produces its value

<br><br>
$x = 2 + 3 = 5$  
$f(4) \rightarrow (16, 2)$  
$x + y + z= 5 + 16 + 2 = 23$







### What does this do?

<p><span class="math display">\[\begin{array}{l}  
\texttt{let}\\  
~~~~f(i) = \texttt{if} ~(i < 2) ~\texttt{then}~ i ~\texttt{else}~ i  *   
f(i - 1) \\  
\texttt{in} \\   
~~~~f(5) \\  
\texttt{end}   
\end{array}\]</span> </p>



In [1]:
def factorial(i):
    if i<2:
        return i
    else:
        return i*factorial(i-1)
    
factorial(5)

120

## Composition [SPARC]




<img src="figures/composition.png" width="50%" />


-   $(e_1, e_2)$: Sequential Composition

    -   Add work and span

-   $(e_1 || e_2)$: Parallel Composition

    -   Add work but **take the maximum span**
    
    

### parallel composition: $(e_1 || e_2)$

- $W(e_1 || e_2) = 1 + W(e_1) + W(e_2)$  
- $S(e_1 || e_2) = 1 + \max(S(e_1), S(e_2))$  

Let's look at the specification and recurrence for Summing List: 



<p><span class="math display">\[\begin{array}{l}  
\mathit{sumList}~a =  
\\   
~~~~\texttt{if}~|a| \leq 1~\texttt{then}  
\\   
~~~~~~~~a  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(l,r) = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(l',r') = (\mathit{sumList}~l \mid\mid{} \mathit{sumList}~r) 
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~l'+r'  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>



### Recurrence
Recurrences are a way to capture the behavior of recursive algorithms.

Key ingredients: 
- Base case ($n = c$): constant time 
- Inductive case ($n > c$): recurse on smaller instance and use output to compute solution


<p><span class="math display">\[\begin{array}{l}  
\mathit{sumList}~a =  
\\   
~~~~\texttt{if}~|a| \leq 1~\texttt{then}  
\\   
~~~~~~~~a  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(l,r) = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(l',r') = (\mathit{sumList}~l \mid\mid{} \mathit{sumList}~r) 
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~l'+r'  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>


#### Previously,

<img src="figures/dag.png" width="70%" />



> **Work** is the number of node: $3n-2 \in O(n)$

> **span** is the longest dependency: $2\log_2(n) \in O(\log n)$


<span style="color:red">**Question**</span>: How can we use recurrence to calculate the work and span?

> The idea is if we can draw the relationship based on the reccurence.


<br>

<br>

<br>


$ \begin{equation}
W(n) = \begin{cases}
  1, & \text{if $n=1$} \\
  2W(n/2) + 1, & \text{otherwise} 
  \end{cases}
\end{equation}$

How do we solve this recurrence to obtain $W(n) = O(n)$? What about the span?

<br>

$ \begin{equation}
S(n) = \begin{cases}
  1, & \text{if $n=1$} \\
  S(n/2) + 1, & \text{otherwise} 
  \end{cases}
\end{equation}$

